In [78]:
from bs4 import BeautifulSoup as bs
import requests
import json
import codecs
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

In [58]:
url = 'https://understat.com/league/Serie_A/2014'

**парсим всю страницу и отдельно нужные теги (здесь `script`)**

In [49]:
page = requests.get(url)
soup = bs(page.text, 'html.parser')
main_text_doc = soup.prettify()

# знаем что нужные данные хранятся в 'script'
# вывожу ТОЛЬКО тэги 'script', чтобы еще раз просмотреть на данные
scripts = soup.find_all('script')
# scripts

**берем данные из `scripts`**

**[механизм](https://stackoverflow.com/questions/55703531/how-to-scrape-json-data-from-an-interactive-chart)**

In [ ]:
# можно так:
# datesData_script = str(soup('script')[1])

In [72]:
# можно так:

# I noticed the data was imbedded in the script tag that started with `var shotsData`
for script in scripts:
    if 'var datesData' in script.text:
# I store that text, then trim off the string on the ends so that it's in a valid json format
        encoded_string = script.text
    
# можно написать универсальную регулярку, для выделения необходимого куска ASCII
        encoded_string  = encoded_string.split("JSON.parse('", 1)[-1]
        encoded_string = encoded_string.rsplit("'),",1)[0]


# Have it ignore the escape characters so it can decode the ascii and be able to use json.loads
        jsonStr = codecs.getdecoder('unicode-escape')(encoded_string)[0]
        jsonObj = json.loads(jsonStr)

SeriaA_datesData = json_normalize(jsonObj)
print(type(SeriaA_datesData))

<class 'pandas.core.frame.DataFrame'>


In [40]:
#берем данные из teamsData 
for script in scripts:
    if 'var teamsData' in script.text:
        encoded_string = script.text
        encoded_string  = encoded_string.split("JSON.parse('", 1)[-1]
        encoded_string = encoded_string.rsplit("');",1)[0]
        jsonStr = codecs.getdecoder('unicode-escape')(encoded_string)[0]
        jsonObj = json.loads(jsonStr)

SeriaA_teamsData = json_normalize(jsonObj)
print(type(SeriaA_teamsData))

# тут данные хранятся совсем в другом порядке, нуждаются в качественном преобразовании
# SeriaA_teamsData['100.history'][0]

<class 'pandas.core.frame.DataFrame'>


In [45]:
# playersData
for script in scripts:
    if 'var playersData' in script.text:
        encoded_string = script.text
        encoded_string  = encoded_string .split("JSON.parse('", 1)[-1]
        encoded_string = encoded_string.rsplit("');",1)[0]
        jsonStr = codecs.getdecoder('unicode-escape')(encoded_string)[0]
        jsonObj = json.loads(jsonStr)

SeriaA_playersData = json_normalize(jsonObj)
print(type(SeriaA_playersData))
# тут понятные данные:
# SeriaA_playersData

<class 'pandas.core.frame.DataFrame'>


## Парсим много страниц в цикле

**создаю список url для парсинга:**

In [64]:
# составляем список чемпионатов
leagues = soup.findAll('li')

leagues_list = []
for league in leagues[-6:]:
#  вытаскиваем текст
    league = league.get_text()
# заменяем пробелы на '_', это нужно для будущей работы
    league = league.replace(' ', '_')
# складываем в список
    leagues_list.append(league)
    
leagues_list

['EPL', 'La_liga', 'Bundesliga', 'Serie_A', 'Ligue_1', 'RFPL']

In [67]:
def create_url_list(url, leagues_list, start_postfix, end_postfix, step):
    """create url list for scrapping data"""   
    url_list = []
    for l in leagues_list:
        for i in range(start_postfix, end_postfix, step):
            new_url = url + l + '/' + str(i)
            url_list.append(new_url)
    return url_list

In [68]:
url = 'https://understat.com/league/'

In [73]:
url_list = create_url_list(url, leagues_list, 2019, 2013, -1)

In [100]:
namer = re.findall(reg, url_list[2])[0] + '_' + re.findall('\d*$', url_list[2])[0]   
namer

'EPL_2017'

In [ ]:
def scrapping_url_list(url_list):
    
    for url in url_list:
        
# это чтобы назвыать документы соответствующим образом
        reg = '(?<=league/)[\w]*'
        namer = re.findall(reg, url)[0] + '_' + re.findall('\d*$', url)[0]    
        
        page = requests.get(url)
        soup = bs(page.text, 'html.parser')
        scripts = soup.find_all('script')
        for script in scripts:
            
            if 'var datesData' in script.text:
                encoded_string = script.text
                encoded_string  = encoded_string.split("JSON.parse('", 1)[-1]
                encoded_string = encoded_string.rsplit("'),",1)[0]
                jsonStr = codecs.getdecoder('unicode-escape')(encoded_string)[0]
                jsonObj = json.loads(jsonStr)
                datesData = json_normalize(jsonObj)
                
            if 'var teamsData' in script.text:
                encoded_string = script.text
                encoded_string  = encoded_string.split("JSON.parse('", 1)[-1]
                encoded_string = encoded_string.rsplit("');",1)[0]
                jsonStr = codecs.getdecoder('unicode-escape')(encoded_string)[0]
                jsonObj = json.loads(jsonStr)
                teamsData = json_normalize(jsonObj)
                
            if 'var playersData' in script.text:
                encoded_string = script.text
                encoded_string  = encoded_string .split("JSON.parse('", 1)[-1]
                encoded_string = encoded_string.rsplit("');",1)[0]
                jsonStr = codecs.getdecoder('unicode-escape')(encoded_string)[0]
                jsonObj = json.loads(jsonStr)
                playersData = json_normalize(jsonObj)
                
        datesData.to_excel(f'understat_{namer}.xlsx', sheet_name=f'datesData_{namer}')
        teamsData.to_excel(f'understat_{namer}.xlsx', sheet_name=f'teamsData_{namer}')
        playersData.to_excel(f'understat_{namer}.xlsx', sheet_name=f'playersData_{namer}')
                


In [103]:
# пока сохраняет неправильно
url = url_list[2]
reg = '(?<=league/)[\w]*'
namer = re.findall(reg, url)[0] + '_' + re.findall('\d*$', url)[0]    

page = requests.get(url)
soup = bs(page.text, 'html.parser')
scripts = soup.find_all('script')
for script in scripts:

    if 'var datesData' in script.text:
        encoded_string = script.text
        encoded_string  = encoded_string.split("JSON.parse('", 1)[-1]
        encoded_string = encoded_string.rsplit("'),",1)[0]
        jsonStr = codecs.getdecoder('unicode-escape')(encoded_string)[0]
        jsonObj = json.loads(jsonStr)
        datesData = json_normalize(jsonObj)

    if 'var teamsData' in script.text:
        encoded_string = script.text
        encoded_string  = encoded_string.split("JSON.parse('", 1)[-1]
        encoded_string = encoded_string.rsplit("');",1)[0]
        jsonStr = codecs.getdecoder('unicode-escape')(encoded_string)[0]
        jsonObj = json.loads(jsonStr)
        teamsData = json_normalize(jsonObj)

    if 'var playersData' in script.text:
        encoded_string = script.text
        encoded_string  = encoded_string .split("JSON.parse('", 1)[-1]
        encoded_string = encoded_string.rsplit("');",1)[0]
        jsonStr = codecs.getdecoder('unicode-escape')(encoded_string)[0]
        jsonObj = json.loads(jsonStr)
        playersData = json_normalize(jsonObj)

datesData.to_excel(f'understat_{namer}.xlsx', sheet_name=f'datesData_{namer}')
teamsData.to_excel(f'understat_{namer}.xlsx', sheet_name=f'teamsData_{namer}')
playersData.to_excel(f'understat_{namer}.xlsx', sheet_name=f'playersData_{namer}')

**экспорт в excel**

*[пример сохранения на разных листах1](https://stackoverflow.com/questions/41215508/using-loop-to-create-excel-sheets-with-dataframe-pandas)


*[пример сохранения на разных листах2](https://xlsxwriter.readthedocs.io/working_with_pandas.html)

In [ ]:
# назначем индексами 'id'
# SeriaA_datesData = SeriaA_datesData.set_index('id', drop=True)

# экспорт
# SeriaA_datesData.to_excel("SeriaA_datesData.xlsx")